In [61]:
'''This repository contains a detailed implementation of the Reinforcement Learning Enviroment class'''
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import *
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy.integrate as integrate
from typing import Any, Callable, Dict, List, Tuple, Union, Optional
from functools import wraps
import os
import random
from abc import ABC, abstractmethod
from collections import deque, namedtuple
import sys
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from OUNoise import OUNoise
from EnforceTyping import EnforceClassTyping, enforce_method_typing, enforce_function_typing
from ParticlesandFields import ClassicalField, ClassicalParticle
from MDPFramework import MDPEnvironment,  LearningAgent
from ActorCriticNetworks import ActorNetwork, CriticNetwork
T.Tensor.ndim = property(lambda self: len(self.shape))

In [62]:
def random_policy(state):
    return np.random.normal(0, 1, size=2)
def no_policy(state):
    return np.zeros((2, ))

**Vector Field**

In [63]:
@dataclass(kw_only=True)
class ElectrostaticField2D(ClassicalField):
  """
  A class used to represent a 2D Electrostatic Field

  Attributes
  ----------
  field_sources: dict
      a formatted string to print out what the animal says
  dimensionality: tuple
      a tuple of the dimensionality of the field  

  Methods
  -------
  dynamics(self, observation_position: np.ndarray, time: float) -> np.ndarray:
      Represents the value of the field at any given point(s) or time. 
  potential(self, observation_position: np.ndarray, time: float) -> float:
      Represents the potential due to the field at a given position and/or time  
  potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray, time: float) -> float:
      Represents the potential difference between two positions at a given time in the vector field   
  gradient(self, observation_position: np.ndarray, time: float) -> float:
      Represents the gradient at a given position and/or time in the vector field 
  curl(self, observation_position: np.ndarray, time: float) -> float:
      Represents the curl at a given position and/or time in the vector field 
  divergence(self, observation_position: np.ndarray, time: float) -> float:
      Represents the divergence at a given position and/or time in the vector field
  """
  field_sources: dict
  dimensionality: tuple = (2,)

  def __call__(self, observation_position: np.ndarray) -> np.ndarray:
      return self.dynamics(observation_position)

  @enforce_method_typing
  def dynamics(self, observation_position: np.ndarray) -> np.ndarray:
      """
      This function outputs the field strength due to field sources experienced at any given point(s) or time. 
      This determines the physics of the field (a 2D Electricstatic Field in this case)

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric field strength vector at the given position.
      """
      assert len(self.field_sources["Particle"]) == len(self.field_sources["Position"]), "The length of particles and fields don't match"
      for field_source, _ in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          assert isinstance(field_source, ClassicalParticle),  "The field source is not a particle"
      coulomb_constant = 8.9875e9  # N*m^2/C^2
      electric_field_vector = np.zeros_like(observation_position)
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_matrices = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vector = observation_position - position_matrices
          displacement_magnitude = np.linalg.norm(displacement_vector, axis=0)
          electric_field_vector += (displacement_vector * field_source.charge) / displacement_magnitude**3
      electric_field_vector = coulomb_constant * electric_field_vector
      return np.round(electric_field_vector, 3)  # N/C or V/m

  @enforce_method_typing
  def potential(self, observation_position: np.ndarray) -> float:
      """
      Calculate the potential (voltage) at a position in the field.

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric potential at the given position.
      """
      assert len(self.field_sources["Particle"]) == len(self.field_sources["Position"]), "The length of particles and fields don't match"
      for field_source, _ in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          assert isinstance(field_source, ClassicalParticle),  "The field source is not a particle"
  
      coulomb_constant = 8.9875e9  # N*m^2/C^2
      electric_potential = 0.0

      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_matrices = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vector = observation_position - position_matrices
          displacement_magnitude = np.linalg.norm(displacement_vector, axis=0)
          electric_potential += field_source.charge / displacement_magnitude

      electric_potential = coulomb_constant * electric_potential
      return np.round(electric_potential, 3)  # V

  @enforce_method_typing
  def potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray) -> float:
    """
    Calculate the potential difference between the initial position and the final position in the field.

    Args:
        initial_position (np.ndarray): The starting position.
        final_position (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    assert initial_position.shape == self.dimensionality, "initial_position has the wrong dimensions"
    assert final_position.shape == self.dimensionality, "final_position has the wrong dimensions"
    PorentialDifference= self.potential(initial_position)- self.potential(final_position)
    return PorentialDifference

  def gradient(self, observation_position: np.ndarray, delta: float= 0.001)->np.ndarray:
    """
    This function returns the derivative of the field at a given point

    Args:
        InitialPosition (np.ndarray): The starting position.
        FinalPosition (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    gradient= np.zeros_like(observation_position)
    for i in range(len(observation_position)):
      di= np.zeros_like(observation_position)
      di[i, ] = di[i, ]+delta
      plusdi= observation_position+ di
      minusdi= observation_position- di
      gradient[i]= (self.dynamics(plusdi)- self.dynamics(minusdi))[i]/ (2* delta)
    return gradient
  
  def curl(self, observation_position: np.ndarray)-> float:
    """
    This function returns the curl of the field at a given point

    Args:
        InitialPosition (np.ndarray): The starting position.
        FinalPosition (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    pass
  
  def divergence(self, observation_position: np.ndarray)-> float:
    """
    This function returns the divergence of the field

    Args:
        InitialPosition (np.ndarray): The starting position.
        FinalPosition (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    pass
  @enforce_method_typing
  def plot_field(self, low_bound= -20, high_bound= 20):
    """
    This funtion plots the 2D electric vector field

    Args:
        InitialPosition (np.ndarray): The starting position.
        FinalPosition (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    observation_position= np.meshgrid(np.linspace(low_bound, high_bound, 25), 
                                    np.linspace(low_bound, high_bound, 25))
    observation_position= np.stack(observation_position)
    xd, yd = self.dynamics(observation_position)
    xd = xd / np.sqrt(xd**2 + yd**2)
    yd = yd / np.sqrt(xd**2 + yd**2)
    color_aara = np.sqrt(xd**2+ yd**2)
    fig, ax = plt.subplots(1,1)
    cp = ax.quiver(observation_position[0],observation_position[1],xd,yd,color_aara)
    fig.colorbar(cp)
    plt.rcParams['figure.dpi'] = 150
    plt.show()
   

**Environment**

In [64]:
@dataclass(kw_only=True)
class ParticleInField(MDPEnvironment):
  """
  A class used to represent a particle in a Field

  Attributes
  ----------
  field: ClassicalField
    The field that the particle is in 
  particle: ClassicalParticle
    The particle that is in the field
  target: np.ndarray 
    The target position of the particle
  distance_weight: float 
    The weight of the distance between the particle and the target
  energy_weight: float 
    The weight of the energy of the particle
  terminal_signal_weight: float 
    The weight of the terminal signal of the particle
  current_time: float 
    The current time of the system

  Methods
  ------- 
  transition_model(self, state: State, action: Any)-> State: 
    Represents the  
  reward_model(self, state: State, action: Any, next_state: State, terminal_signal: bool)-> float:
    Represents the reward of the system
  is_terminal_condition(self, state: State)-> bool: 
    Represents the terminal condition of the system
  transition_step(self, state: State, action: Any)-> tuple[float, State, bool]: 
    Represents the transition step of the system
  sample_trajectory(self, runtime: float)-> list[State]: 
    Samples a trajectory of the system
  trajectory_value(self, trajectory: list[State])-> float: 
    Represents the value of the trajectory
  reset(self): 
    Resets the system

  """
  field: ClassicalField
  particle: ClassicalParticle
  target: np.ndarray # m
  distance_weight: float= 1.0
  energy_weight: float= -1.0
  terminal_signal_weight: float= -1000.0
  current_time: float = 0.0# s
  
  @dataclass 
  class State(EnforceClassTyping):
    '''This class represents the state of the Agent with its position, velocity and the Field Strength if experiences at its position. 
    These are parameters the agent is able to observe, they uniquely define the state of the agent.'''
    position: np.ndarray # m
    velocity: np.ndarray #m/s
    
    def vector(self):
      return np.concatenate([self.position, self.velocity])
    
  initial_state: State = None
  current_state: State = None

  def __post_init__(self):
    if self.initial_state is None:
        self.initial_state= self.random_state()
    self.current_state= self.initial_state
  @enforce_method_typing
  def state_dynamics(self, state: np.ndarray, time: float, control_force: np.ndarray):
    """
    Compute the dynamics of the particle's state.

    Parameters:
    state (np.ndarray): The current state of the particle [x, y, vx, vy].
    time (float): The current time.
    control_force (np.ndarray): The external control force applied to the particle.

    Returns:
    np.ndarray: The derivative of the state [vx, vy, ax, ay].
    """
    velocity = state[2:]
    acceleration = (self.particle.charge * self.field.dynamics(state[:2]) + control_force) / self.particle.mass
    return np.concatenate((velocity, acceleration))
  
  @enforce_method_typing
  def transition_model(self, state: State, action: np.ndarray= np.array([0.0, 0.0]), time_interval:float= 1.0)-> State:
    """
    Computes the next state of the system after applying a constant force for a given time interval.

    Args:
        state (State): The current state of the system.
        action (np.ndarray, optional): The constant force to apply. Defaults to [0.0, 0.0].
        time_interval (float, optional): The time interval to apply the force for. Defaults to 1.0.

    Returns:
        State: The next state of the system.
    """
    t_span = [self.current_time, self.current_time + time_interval]
    next_state_vector = integrate.odeint(self.state_dynamics, state.vector(), t_span, args=(action,))[-1]
    next_position = next_state_vector[:2]
    next_velocity = next_state_vector[2:]
    return self.State(next_position, next_velocity)
  @enforce_method_typing
  def reward_model(self, state: State, action: np.ndarray, next_state: State, terminal_signal: bool) -> float:
      """
      Computes the reward for the agent given a state transition.

      The reward is a weighted sum of three components:
      1. Distance gained towards the target
      2. Energy consumed during the transition
      3. Terminal signal (e.g. reaching the target or running out of energy)

      Args:
          state: The current state of the agent
          action: The action taken by the agent
          next_state: The resulting state after taking the action
          terminal_signal: A boolean indicating whether the episode has terminated

      Returns:
          float: The reward value
      """
      distance_gained = np.linalg.norm(state.position - self.target) - np.linalg.norm(next_state.position - self.target)
      energy_consumed = self.particle.charge * self.field.potential(state.position, next_state.position)
      reward = (
          self.distance_weight * distance_gained
          + self.energy_weight * energy_consumed
          + self.terminal_signal_weight * int(terminal_signal)
      )
      return reward
  
  @enforce_method_typing
  def is_terminal_condition(self, state: State) -> bool:
      """
      Checks if the state is outside the viable learning region of the environment.

      Args:
          state (State): The current state of the environment.

      Returns:
          bool: True if the state is outside the viable learning region, False otherwise.
      """
      x_bound = -10.0 <= state.position[0] <= 10.0
      y_bound = -10.0 <= state.position[1] <= 10.0
      velocity_bound = np.linalg.norm(state.velocity) < 10.0

      return not (x_bound and y_bound and velocity_bound)
  @enforce_method_typing
  def transition_step(
      self, 
      state: State = current_state, 
      action: np.ndarray = np.array([0.0, 0.0]), 
      time_interval: float = 1.0
  ) -> Tuple[State, float, bool]:
      """
      Simulates a single time step of the environment.

      Args:
          state (State): The current state of the environment. Defaults to current_state.
          action (np.ndarray): The action to take in the environment. Defaults to [0.0, 0.0].
          time_interval (float): The time interval for the simulation. Defaults to 1.0.

      Returns:
          Tuple[State, float, bool]: A tuple containing the next state, the reward, and a terminal signal.
      """
      next_state = self.transition_model(state, action, time_interval=time_interval)
      terminal_signal = self.is_terminal_condition(next_state)
      reward = self.reward_model(state, action, next_state, terminal_signal)
      return next_state, reward, terminal_signal

  def random_state(self) -> State:
      """
      Generates a random state within the viable learning region.

      Returns:
          State: A random state within the viable learning region
      """
      position = np.random.uniform(-10.0, 10.0, size=self.field.dimensionality)
      velocity = np.zeros_like(position)
      return self.State(position, velocity)

  @enforce_method_typing
  def sample_trajectory(
      self, 
      runtime: float, 
      policy: Optional[Callable[[np.ndarray], np.ndarray]] = no_policy, 
      time_interval: float = 0.1
  ) -> Tuple[List[Any], List[np.ndarray], List[float]]:
      """
      Generates a random state trajectory within the viable learning region.

      Args:
      - runtime: The total time for the trajectory.
      - policy: An optional policy function that takes a state and returns an action.
      - time_interval: The time interval between each state in the trajectory.

      Returns:
      - A tuple containing the state trajectory, action trajectory, and time points.
      """
      time_points = [0]
      state = self.current_state
      state_trajectory = []
      action_trajectory = []

      while time_points[-1] < runtime:
          state_trajectory.append(state)
          action = policy(state.vector())
          action_trajectory.append(action)
          state = self.transition_model(state, action, time_interval=time_interval)
          time_points.append(time_points[-1] + time_interval)

      return state_trajectory, action_trajectory, time_points

  @enforce_method_typing
  def plot_trajectory(self, state_trajectory: list, time: list) -> None:
      """
      Plot the trajectory of states over time.

      Args:
      - state_trajectory: A list of State objects representing the trajectory.
      - time: A list of time points corresponding to each state in the trajectory.

      Returns:
      - None (plots the trajectory)
      """
      positions = np.array([state.position for state in state_trajectory])
      velocities = np.array([state.velocity for state in state_trajectory])

      plt.figure(figsize=(8, 8))
      plt.plot(positions[:, 0], positions[:, 1], label='Trajectory')
      plt.scatter(positions[0, 0], positions[0, 1], c='k', marker='o', label='Start')
      plt.scatter(positions[-1, 0], positions[-1, 1], c='r', marker='*', label='End')

      plt.xlim(-10, 10)
      plt.ylim(-10, 10)
      plt.grid(True)
      plt.legend()
      plt.show()

  @enforce_method_typing
  def trajectory_value(self, state_trajectory: list, action_trajectory: list, time_points: list) -> float:
      """
      Compute the value of a trajectory.

      Args:
      - state_trajectory: A list of State objects representing the trajectory.
      - action_trajectory: A list of actions corresponding to each state in the trajectory.
      - time_points: A list of time points corresponding to each state in the trajectory.

      Returns:
      - The value of the trajectory.
      """
      total_value = 0.0
      time_interval = (time_points[-1] - time_points[0]) / len(time_points)

      for state, action in zip(state_trajectory, action_trajectory):
          position_error = np.linalg.norm(state.position - self.target)
          action_magnitude = np.linalg.norm(action)
          total_value += (position_error + action_magnitude) * time_interval

      return total_value
  
  def reset(self) -> None:
      """
      Resets the current state to the initial state and sets the current time to 0.0.
      """
      self.current_state = self.initial_state
      self.current_time = 0.0


**Critic Network**

In [69]:
@dataclass
class DDPGAgent(LearningAgent):
    actor_learning_rate: float
    critic_learning_rate: float
    observation_dimensions: tuple
    soft_update_rate: float
    environment: MDPEnvironment
    discount_rate=0.99
    action_dimensions= 2
    max_size=1000
    layer1_size=20
    layer2_size=15
    batch_size=16
    ControlInterval= 0.5

    def __post_init__(self):
        self.Actor = ActorNetwork(self.actor_learning_rate, self.observation_dimensions, self.layer1_size, self.layer2_size, n_actions=self.action_dimensions, name='Actor')
        self.Critic = CriticNetwork(self.critic_learning_rate, self.observation_dimensions, self.layer1_size, self.layer2_size, n_actions=self.action_dimensions, name='Critic')
        self.TargetActor = ActorNetwork(self.actor_learning_rate, self.observation_dimensions, self.layer1_size, self.layer2_size, n_actions=self.action_dimensions, name='TargetActor')
        self.TargetCritic = CriticNetwork(self.critic_learning_rate, self.observation_dimensions, self.layer1_size, self.layer2_size, n_actions=self.action_dimensions, name='TargetCritic')
        for target_param, param in zip(self.TargetActor.parameters(), self.Actor.parameters()):
            target_param.data.copy_(param.data)
        for target_param, param in zip(self.TargetCritic.parameters(), self.Critic.parameters()):
            target_param.data.copy_(param.data) 
        self.memory = deque(maxlen=self.max_size)
        self.noise = OUNoise(mu=np.zeros(self.action_dimensions))
        self.ControlInterval= self.ControlInterval# Acttion duration= conrol interval
        self.update_network_parameters(SoftUpdateRate=1)

    def act(self, observation):
        self.Actor.eval()
        observation = T.tensor(observation, dtype=T.float).to(self.Actor.device)
        mu = self.Actor.forward(observation).to(self.Actor.device)
        mu_prime = 1e-7* (mu + T.tensor(self.noise(), dtype=T.float).to(self.Actor.device))
        self.Actor.train()
        return mu_prime.cpu().detach()

    def observe(self, State= None):
        if State is None:
          State= self.environment.CurrentState   
        if type(State) == tuple:
            Observation= [0]*len(State)
            for i in range(len(State)):
              Observation[i]= self.Observe(State[i])
            Observation= T.stack(Observation)
        else:
            Observation= T.cat([State.position,
                                State.velocity])
        return Observation
       
    def learn(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, next_states, rewards, dones = zip(*batch)

        state = self.Observe(states).to(self.Critic.device)
        action = T.stack(actions).to(self.Critic.device)
        reward = T.tensor(rewards, dtype=T.float).unsqueeze(1).to(self.Critic.device)
        new_state = self.Observe(next_states).to(self.Critic.device)
        done = T.tensor(dones, dtype=T.float).unsqueeze(1).to(self.Critic.device)
        
        self.TargetActor.eval()
        self.TargetCritic.eval()
        self.Critic.eval()
        
        target_actions = self.TargetActor.forward(new_state)
        Critic_value_ = self.TargetCritic.forward(new_state, target_actions) 
        q_expected = self.Critic.forward(state, action)
        q_targets = reward + self.discount_rate * Critic_value_ * (1 - done)

        Critic_loss = nn.MSELoss()(q_expected, q_targets.detach())
        self.Critic.train()
        self.Critic.optimizer.zero_grad()
        Critic_loss.backward()
        self.Critic.optimizer.step()

        self.Actor.eval()
        self.Critic.eval()

        mu = self.Actor.forward(state)
        Actor_loss = -self.Critic.forward(state, mu)

        Actor_loss = T.mean(Actor_loss)
        self.Actor.train()
        self.Actor.optimizer.zero_grad()
        Actor_loss.backward()
        self.Actor.optimizer.step()

        self.update_network_parameters()

    def Run(self, RunDuration: float, Resolution: int=100):
        Path= []
        State= self.environment.InitialState
        Time= 0
        for _ in range(Resolution):
            Path.append(State.Position)
            action= self.choose_action(self.Observe(State))
            State= self.environment.TransitionModel(State, action)
            Time += (RunDuration/Resolution)
        return Path
    
    def PlotRun(self, RunDuration: float):
        Path= self.Run(RunDuration)
        Path= T.stack(Path)
        Path= Path.transpose(dim0=0, dim1=1)
        # print(Path)
        t=  T.arange(0, RunDuration)
        plt.plot(Path[0], Path[1])
        plt.plot(Path[0][0], Path[1][0], 'ko')
        plt.plot(Path[0][-1], Path[1][-1], 'r*')
        plt.xlim(-10,10)
        plt.ylim(-10,10)
        plt.grid(True)
        plt.show()
    
    def update_network_parameters(self, SoftUpdateRate=None):
        if SoftUpdateRate is None:
            SoftUpdateRate = self.soft_update_rate

        Critic_state_dict = dict(self.Critic.named_parameters())
        Actor_state_dict = dict(self.Actor.named_parameters())
        TargetCritic_dict = dict(self.TargetCritic.named_parameters())
        TargetActor_dict = dict(self.TargetActor.named_parameters())

        for name in Critic_state_dict:
            Critic_state_dict[name] = SoftUpdateRate*Critic_state_dict[name].clone() + (1-SoftUpdateRate)*TargetCritic_dict[name].clone()
        self.TargetCritic.load_state_dict(Critic_state_dict)

        for name in Actor_state_dict:
            Actor_state_dict[name] = SoftUpdateRate*Actor_state_dict[name].clone() + (1-SoftUpdateRate)*TargetActor_dict[name].clone()
        self.TargetActor.load_state_dict(Actor_state_dict)

        """
        #Verify that the copy assignment worked correctly
        TargetActor_params = self.TargetActor.named_parameters()
        TargetCritic_params = self.TargetCritic.named_parameters()

        Critic_state_dict = dict(TargetCritic_params)
        Actor_state_dict = dict(TargetActor_params)
        print('\nActor Networks', tau)
        for name, param in self.Actor.named_parameters():
            print(name, T.equal(param, Actor_state_dict[name]))
        print('\nCritic Networks', tau)
        for name, param in self.Critic.named_parameters():
            print(name, T.equal(param, Critic_state_dict[name]))
        input()
        """
    
    def save_models(self):
        self.Actor.save_checkpoint()
        self.TargetActor.save_checkpoint()
        self.Critic.save_checkpoint()
        self.TargetCritic.save_checkpoint()

    def load_models(self):
        self.Actor.load_checkpoint()
        self.TargetActor.load_checkpoint()
        self.Critic.load_checkpoint()
        self.TargetCritic.load_checkpoint()
 


In [70]:
def DDPGAlgorithm(self):
    score_history = []
    for i in range(10):
        self.LearningEnvironment.CurrentState = self.LearningEnvironment.initial_state
        IsDone = False
        score = 0
        for _ in range(50):
            Observation= self.Observe()
            Action = self.choose_action(Observation) 
            new_state, Reward, IsDone= self.LearningEnvironment.Step(self.LearningEnvironment.CurrentState, Action, self.ControlInterval) 
            self.memory.append((self.LearningEnvironment.CurrentState, Action, new_state, Reward, int(IsDone)))
            self.learn()
            score += Reward
            # print(Reward)
            self.LearningEnvironment.CurrentState = new_state
        score_history.append(score)
    plt.plot(score_history)
    return score_history
    

In [71]:
negative_charge= ClassicalParticle(mass=1.0, charge= -1e-9)
positive_charge= ClassicalParticle(mass=1.0, charge= 1e-9)
sources = {"Particle": [negative_charge, positive_charge],
           "Position": [np.array([1.0, 1.0]), np.array([-1.0, 1.0])]}
testElectricField= ElectrostaticField2D(field_sources=sources)
PositiveChargeInElectricField= ParticleInField(field=testElectricField, 
                                               particle=positive_charge, 
                                               target=np.array([0.0, 0.0]))

In [72]:
agent = DDPGAgent(0.0025, 0.0025, 4, 0.001, PositiveChargeInElectricField)
agent.DDPGAlgorithm()
agent.PlotRun(100)

TypeError: DDPGAgent.__init__() missing 1 required positional argument: 'soft_update_rate'